In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import json
import pymongo

In [6]:
!which chromedriver 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [36]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.mars_db
collection = db.items

In [37]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

For InSight, Dust Cleanings Will Yield New Science
Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


In [38]:
image_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url_featured)

In [39]:
html_image = browser.html
soup = BeautifulSoup(html_image, 'html.parser')
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + featured_image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17843-1920x1200.jpg'

In [40]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [41]:
html_weather = browser.html
soup = BeautifulSoup(html_weather, 'html.parser')
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')
for tweet in latest_tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 157 (2019-05-06) low -100.2ºC (-148.4ºF) high -18.4ºC (-1.2ºF)
winds from the W at 4.1 m/s (9.2 mph) gusting to 14.6 m/s (32.7 mph)
pressure at 7.40 hPapic.twitter.com/R6BliV8xpj


In [42]:
facts_url = 'http://space-facts.com/mars/'
mars_facts = pd.read_html(facts_url)
mars_df = mars_facts[0]
mars_df.columns = ['Description','Value']
mars_df.set_index('Description', inplace=True)
mars_df.to_html()
data = mars_df.to_dict(orient='records')  # Here's our added param..
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [43]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [44]:
html_hemispheres = browser.html

soup = BeautifulSoup(html_hemispheres, 'html.parser')

items = soup.find_all('div', class_='item')

hemisphere_image_urls = []

hemispheres_main_url = 'https://astrogeology.usgs.gov'

for i in items: 
    title = i.find('h3').text
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    browser.visit(hemispheres_main_url + partial_img_url)
    partial_img_html = browser.html
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]